In [6]:
import pandas as pd
import numpy as np

import PTN_to_event_network as PTN_to_event_network
import get_paths as get_paths
import tdm_b as tdm_b
import forward_CPM as forward_CPM

import importlib

importlib.reload(get_paths)

<module 'get_paths' from '/Users/giovanniinchiostro/public_transportation/get_paths.py'>

In [30]:
MIN_TRANSFER = [0,0,0,0,0]
# min time to transfer at each station

VEHICLE_PATHS = [
        [(0,1),(1,0)],
        [(2,1),(1,3)],
]
SCHEDULE = [
    [0,5,7,10],
    [0,4,6,10]
]

MIN_TIMES = [
    [4,1,3],
    [3,1,4]
]

E_DEL = [0,0,0,2,5,3,0,2] #What is the length of e_del? total length of schedule

In [29]:
EAN, columns,rows, edge_to = PTN_to_event_network.PTN_to_event_network(MIN_TRANSFER, VEHICLE_PATHS, SCHEDULE, MIN_TIMES)

In [18]:
paths, weighting, changing, driving = get_paths.get_paths(EAN, columns,rows)

In [23]:
WEIGHTS = np.random.rand(len(paths))
E_DEL = [0,0,0,0,1,1,0,2]

In [25]:
maintained_paths = tdm_b.tdm_b(EAN, columns, rows, E_DEL, WEIGHTS)

In [27]:
maintained_paths

[((1, 1, -1), (1, 3, 1)),
 ((0, 0, -1), (0, 1, 1)),
 ((0, 1, -1), (0, 0, 1)),
 ((1, 2, -1), (1, 1, 1)),
 ((1, 2, -1), (1, 1, 1), (0, 1, -1), (0, 0, 1)),
 ((0, 0, -1), (0, 1, 1), (1, 1, -1), (1, 3, 1))]

In [31]:
edge_to_trimmed = forward_CPM.keep_critcal_connections(maintained_paths, edge_to)

In [32]:
edge_to_trimmed

{-1: [((0, 1), (1, 6)), ((1, 5), (0, 2))],
 0: [[1, 4]],
 1: [[2, 1], [6, 0]],
 2: [[3, 3]],
 4: [[5, 3]],
 5: [[6, 1], [2, 0]],
 6: [[7, 4]]}

In [33]:
forward_CPM.forward_CPM(SCHEDULE,E_DEL, edge_to_trimmed)

[0, 5, 8, 12, 5, 8, 9, 13]

## Example 2

In [60]:
MIN_TRANSFER = [0,1,0,0]
#assuming paths are fed in so that there are no loops(ok if whole path is a loop)
#each list is for each vehicle sequentially, and the tuples are edges in the path.
VEHICLE_PATHS = [
        [(0,1),(1,0)],
        [(2,1),(1,2)],
        [(3,1),(1,3)]
]
#assuming sceudels (and minimum times) are in-order with the paths
SCHEDULE = [
    [0,5,10,150],
    [0,7,8,100],
    [0,4,6,100]
]

MIN_TIMES = [
    [4,1,3],
    [3,1,2],
    [3,1,4]
]

In [61]:
EAN, columns,rows, edge_to = PTN_to_event_network.PTN_to_event_network(MIN_TRANSFER, VEHICLE_PATHS, SCHEDULE, MIN_TIMES)

In [62]:
paths, weighting, changing, driving = get_paths.get_paths(EAN, columns,rows)

In [66]:
WEIGHTS = np.random.rand(len(paths))
length_E_DEL = sum(len(_) for _ in SCHEDULE)
E_DEL = np.random.randint(3, size=length_E_DEL)
E_DEL

array([0, 2, 2, 0, 1, 2, 1, 1, 1, 2, 1, 1])

In [68]:
maintained_paths = tdm_b.tdm_b(EAN,columns,rows, E_DEL,WEIGHTS )

In [69]:
edge_to_trimmed = forward_CPM.keep_critcal_connections(maintained_paths, edge_to)

In [73]:
forward_CPM.forward_CPM(SCHEDULE,E_DEL, edge_to_trimmed)

[np.int64(0),
 np.int64(7),
 np.int64(12),
 np.int64(150),
 np.int64(1),
 np.int64(9),
 np.int64(10),
 np.int64(101),
 np.int64(1),
 np.int64(6),
 np.int64(8),
 np.int64(101)]